In [ ]:
import pandas as pd
import numpy as np
import torch as pt
import cv2 as cv

import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import LFWPairs
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
training = LFWPairs('./data/train', split='train', download=True, transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
]))

validating = LFWPairs('./data/validate', download=True, transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
]))

testing = LFWPairs('./data/test', split='test', download=True, transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
]))

In [ ]:
training_set = DataLoader(training, batch_size=64, pin_memory=True)
validation_set = DataLoader(validating, batch_size=64)
testing_set = DataLoader(testing, batch_size=64)

In [ ]:
class ContrastiveLoss(nn.Module): 
    
    def __init__(self, margin=2.0):
        super().__init__(ContrastiveLoss, self)
        self.margin = margin
    
    def forward(self, input1, input2, label):
        pairwise_dist = F.pairwise_distance(input1, input2)
        distance = (1 - label) * torch.pow(pairwise_dist, 2) + \
            (label) * torch.pow(torch.clamp(margin - pairwise_dist, min=0), 2)
        return distance
        

In [ ]:
class SiameseNetwork(nn.Module):
    
    def __init__(self):
        self.distance = ContrastiveLoss()
        
        self.cnn = nn.Sequential(
            
            
            
        )